# Parameter Management

Once we have chosen an architecture
and set our hyperparameters,
we proceed to the training loop,
where our goal is to find parameter values
that minimize our loss function.
After training, we will need these parameters
in order to make future predictions.
Additionally, we will sometimes wish
to extract the parameters
perhaps to reuse them in some other context,
to save our model to disk so that
it may be executed in other software,
or for examination in the hope of
gaining scientific understanding.

Most of the time, we will be able
to ignore the nitty-gritty details
of how parameters are declared
and manipulated, relying on deep learning frameworks
to do the heavy lifting.
However, when we move away from
stacked architectures with standard layers,
we will sometimes need to get into the weeds
of declaring and manipulating parameters.
In this section, we cover the following:

* Accessing parameters for debugging, diagnostics, and visualizations.
* Sharing parameters across different model components.


In [30]:
import torch
from torch import nn

(**We start by focusing on an MLP with one hidden layer.**)


In [31]:
net = nn.Sequential(nn.LazyLinear(8),
                    nn.ReLU(),
                    nn.LazyLinear(1))

X = torch.rand(size=(2, 4))
net(X).shape

torch.Size([2, 1])

## [**Parameter Access**]
:label:`subsec_param-access`

Let's start with how to access parameters
from the models that you already know.


When a model is defined via the `Sequential` class,
we can first access any layer by indexing
into the model as though it were a list.
Each layer's parameters are conveniently
located in its attribute.


We can inspect the parameters of the second fully connected layer as follows.


In [32]:
net[2].state_dict()

OrderedDict([('weight',
              tensor([[-0.2202,  0.2929,  0.2868, -0.3273, -0.2131, -0.0794,  0.1848, -0.0617]])),
             ('bias', tensor([0.1436]))])

We can see that this fully connected layer
contains two parameters,
corresponding to that layer's
weights and biases, respectively.


### [**Targeted Parameters**]

Note that each parameter is represented
as an instance of the parameter class.
To do anything useful with the parameters,
we first need to access the underlying numerical values.
There are several ways to do this.
Some are simpler while others are more general.
The following code extracts the bias
from the second neural network layer, which returns a parameter class instance, and
further accesses that parameter's value.


In [33]:
type(net[2].bias), net[2].bias.data

(torch.nn.parameter.Parameter, tensor([0.1436]))

Parameters are complex objects,
containing values, gradients,
and additional information.
That is why we need to request the value explicitly.

In addition to the value, each parameter also allows us to access the gradient. Because we have not invoked backpropagation for this network yet, it is in its initial state.


In [34]:
net[2].weight.grad == None

True

### [**All Parameters at Once**]

When we need to perform operations on all parameters,
accessing them one-by-one can grow tedious.
The situation can grow especially unwieldy
when we work with more complex, e.g., nested, modules,
since we would need to recurse
through the entire tree to extract
each sub-module's parameters. Below we demonstrate accessing the parameters of all layers.


In [35]:
[(name, param.shape) for name, param in net.named_parameters()]

[('0.weight', torch.Size([8, 4])),
 ('0.bias', torch.Size([8])),
 ('2.weight', torch.Size([1, 8])),
 ('2.bias', torch.Size([1]))]

## [**Tied Parameters**]

Often, we want to share parameters across multiple layers.
Let's see how to do this elegantly.
In the following we allocate a fully connected layer
and then use its parameters specifically
to set those of another layer.
Here we need to run the forward propagation
`net(X)` before accessing the parameters.


In [36]:
# We need to give the shared layer a name so that we can refer to its
# parameters
shared = nn.LazyLinear(8)
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.LazyLinear(1))

net(X)
# Check whether the parameters are the same
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# Make sure that they are actually the same object rather than just having the
# same value
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


This example shows that the parameters
of the second and third layer are tied.
They are not just equal, they are
represented by the same exact tensor.
Thus, if we change one of the parameters,
the other one changes, too.


You might wonder,
when parameters are tied
what happens to the gradients?
Since the model parameters contain gradients,
the gradients of the second hidden layer
and the third hidden layer are added together
during backpropagation.


## Summary

We have several ways of accessing and tying model parameters.


## Exercises

1. Use the `NestMLP` model defined in :numref:`sec_model_construction` and access the parameters of the various layers.
1. Construct an MLP containing a shared parameter layer and train it. During the training process, observe the model parameters and gradients of each layer.
1. Why is sharing parameters a good idea?


[Discussions](https://discuss.d2l.ai/t/57)


In [46]:
#  Exercise 1
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.LazyLinear(64), nn.ReLU(),
                                 nn.LazyLinear(32), nn.ReLU())
        self.linear = nn.LazyLinear(16)

    def forward(self, X):
        return self.linear(self.net(X))

net = NestMLP()
blah = net(torch.rand((2,20)))
layer_idx = 0
for name, module in net.named_modules():
  if isinstance(module, (nn.Linear, nn.LazyLinear)):
    print(f"Layer {layer_idx} ({name}):")
    print(f"  weight: {module.weight.data}")
    print(f"  bias:   {module.bias.data}")
    layer_idx += 1

Layer 0 (net.0):
  weight: tensor([[-0.0413,  0.2025, -0.1952,  ..., -0.1712, -0.1561, -0.1739],
        [-0.0292, -0.1820,  0.1680,  ..., -0.2107, -0.0515, -0.1078],
        [-0.1216,  0.1733, -0.1661,  ..., -0.2079, -0.1591,  0.1051],
        ...,
        [-0.0393, -0.1211,  0.1198,  ..., -0.0392,  0.0152, -0.0955],
        [ 0.0720,  0.1806, -0.1029,  ...,  0.0282,  0.0698, -0.2081],
        [ 0.0192,  0.1625, -0.0339,  ...,  0.1951,  0.1769, -0.0451]])
  bias:   tensor([-0.1494, -0.0784, -0.1820,  0.1754, -0.1542, -0.1477,  0.0380, -0.0737,
        -0.1642, -0.1109, -0.1341, -0.1247,  0.1313,  0.0531, -0.1253,  0.0227,
        -0.0588, -0.0126,  0.0402, -0.0265, -0.2232, -0.1015,  0.0577, -0.1447,
        -0.2141,  0.0026, -0.0400,  0.2079, -0.1236,  0.2024,  0.0444, -0.0209,
        -0.0664, -0.1077, -0.2127, -0.0584,  0.0626, -0.0530,  0.1068, -0.0477,
        -0.1278,  0.0758,  0.1456,  0.1988,  0.0300,  0.1091, -0.0805,  0.0616,
         0.1599,  0.1833, -0.1307, -0.1655,  0.11

In [38]:
# Exercise 2
class SharedMLP(nn.Module):
  def __init__(self):
    super().__init__()
    shared = nn.LazyLinear(8)
    self.net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.LazyLinear(1))

  def forward(self, X):
    return self.net(X)

In [39]:
net = SharedMLP()
X = torch.rand(20, 20)         # input
y = torch.rand(20, 1)          # target

criterion = nn.MSELoss()       # loss function
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

for epoch in range(100):
    optimizer.zero_grad()
    y_pred = net(X)            # forward pass
    loss = criterion(y_pred, y)
    loss.backward()            # backward pass
    optimizer.step()           # update weights

    print(f"Epoch {epoch}: loss = {loss.item():.4f} --> {[(name, param.shape) for name, param in net.named_parameters()]}")

Epoch 0: loss = 0.9186 --> [('net.0.weight', torch.Size([8, 20])), ('net.0.bias', torch.Size([8])), ('net.2.weight', torch.Size([8, 8])), ('net.2.bias', torch.Size([8])), ('net.6.weight', torch.Size([1, 8])), ('net.6.bias', torch.Size([1]))]
Epoch 1: loss = 0.8746 --> [('net.0.weight', torch.Size([8, 20])), ('net.0.bias', torch.Size([8])), ('net.2.weight', torch.Size([8, 8])), ('net.2.bias', torch.Size([8])), ('net.6.weight', torch.Size([1, 8])), ('net.6.bias', torch.Size([1]))]
Epoch 2: loss = 0.8328 --> [('net.0.weight', torch.Size([8, 20])), ('net.0.bias', torch.Size([8])), ('net.2.weight', torch.Size([8, 8])), ('net.2.bias', torch.Size([8])), ('net.6.weight', torch.Size([1, 8])), ('net.6.bias', torch.Size([1]))]
Epoch 3: loss = 0.7933 --> [('net.0.weight', torch.Size([8, 20])), ('net.0.bias', torch.Size([8])), ('net.2.weight', torch.Size([8, 8])), ('net.2.bias', torch.Size([8])), ('net.6.weight', torch.Size([1, 8])), ('net.6.bias', torch.Size([1]))]
Epoch 4: loss = 0.7558 --> [('ne